# Demo Python/Jupyter analysis of MachineMetrics high-frequency predictive data
This customer is manufacturing automotive parts on a mill. In the time period covered by our example data set, the customer reported that 17 parts had to be scrapped due to a tool going bad without the operator having noticed. Besides the lost material, that led to about half an hour of wasted machine time. 


### *Could we have spotted this problem earlier using high-frequency data monitoring?*

## 0. Download and unzip the data set, if you don't already have it!
* Only run this cell if you didn't already download/unzip.
* Place the unzipped CSVs into the same folder as this notebook.

In [ ]:
from urllib.request import urlopen
from zipfile import ZipFile
print("Downloading from S3...")
zipname = 'hf_python_files.zip'
zipresp = urlopen(f'https://machinemetrics-public.s3-us-west-2.amazonaws.com/ds/{zipname}')
with open(zipname, 'wb') as zipfile:
    zipfile.write(zipresp.read())
print("Unzipping...")
with ZipFile(zipname) as zipfile:
    zipfile.extractall()

## 1. Import packages, customize plot settings

In [ ]:
import pandas as pd
from IPython.display import display, clear_output
from time import time, sleep

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 100
matplotlib.rcParams['axes.titlesize'] = 20
matplotlib.rcParams['axes.titleweight'] = 'bold'
matplotlib.rcParams['axes.labelsize'] = 16
matplotlib.rcParams['legend.fontsize'] = 16

## 2. Load the data set
* Consists of a dense high-frequency (kHz) set and a sparse set consisting of cleaned-up partcount and tool use period boundaries.
* Each covers a ~3h45m period.
* 132 parts, typical cycle time ~90s, 3 distinct tool use periods per part.

In [ ]:
df_hf = pd.read_csv('hf2.csv', parse_dates=True, index_col='timestamp')
df_pt = pd.read_csv('partsTools.csv', parse_dates=True, index_col='timestamp')

print("Raw kHz high-frequency data set:")
display(df_hf)
print("Precleaned partcount/tool boundaries data set:")
display(df_pt)

## 3. Data grooming:
* Simplify column names
* Cut out low-activity periods (pauses between part cycles, dowtime, etc)
* Use the partcount and tool dataframe as a lookup table for *rapidly* slicing the HF dataframe

In [ ]:
df_hf = (
    df_hf
    .rename(columns={'SPINDLE_1_load':'load', 'SPINDLE_1_motor_speed':'speed'})
    .query('load > 2 or abs(speed) > 2')
)
print("The groomed kHz time series:")
display(df_hf)

df_pt = (
    df_pt
    .reset_index()
    .set_index(['partcount', 'T'])
)
all_parts_tools = df_pt.index[:-3]  # the last few steps over-run the kHz data
print("The lookup table for parts and tools:")
display(df_pt)

def get_part_tool(partcount, T):
    """
    Get a specific partcount & tool use period from the HF time series using efficient Pandas time slicing
    """
    if (partcount, T) not in df_pt.index:
        return None
    start_timestamp = df_pt.at[(partcount, T), 'timestamp']
    end_timestamp =  df_pt.shift(-1).at[(partcount, T), 'timestamp']
    return df_hf.loc[start_timestamp:end_timestamp] if pd.notna(end_timestamp) else None

## 4. Test: Grab a specific (partcount, tool) period using the lookup table

In [ ]:
print(df_pt.loc[(45,1)]['timestamp'], 'thru', df_pt.loc[(45,2)]['timestamp'])
dft = get_part_tool(45, 1)
display(dft)
dft['load'].plot()
plt.show()

## 5. First exploratory: What do the raw signals look like before/during the fault?

In [ ]:
all_tools = all_parts_tools.get_level_values(1).unique()

def plot_parts(partcounts, metric, tools=all_tools, **plt_kwargs):
    """
    Custom plotting function for a given list of parts and a given HF metric
    Constructs one plot per tool use period, and overlays multiple parts on each plot
    """
    partcounts = partcounts if hasattr(partcounts, '__iter__') else [partcounts]
    fig, axs = plt.subplots(1, len(tools), figsize=(24, 5))
    axs[0].set_ylabel(metric, fontsize=20)
    for T, ax in zip(tools, axs):
        for partcount in partcounts:
            label = f'part {partcount}'
            if (partcount, T) in all_parts_tools:
                get_part_tool(partcount, T)[metric].plot(label=label, ax=ax, use_index=False, **plt_kwargs)
            else:
                ax.plot([], [], label=label)
        ax.set_title(f'T = {T}')
        ax.set_xlabel(f'milliseconds')
        plt.gca().relim()
    plt.legend()
    plt.show()

test_parts = [0, 1, 121, 122]
plot_parts(test_parts, 'load')
plot_parts(test_parts, 'speed')

## 6. Any clues in simple integrated time series features? First, build some and put them into a dataframe.

In [ ]:
feature_rows = []
previous_partcount = 0
for partcount, T in all_parts_tools:
    if partcount != previous_partcount:
        print(partcount, end=' ')
        previous_partcount = partcount
    dft = get_part_tool(partcount, T)
    feature_rows.append({
        'duration':len(dft),
        'load_integral':dft['load'].sum(),
        })
print()
df_features = pd.DataFrame(index=all_parts_tools, data=feature_rows)
display(df_features)

## 7. Plot some (normalized) features for our sample of parts

In [ ]:
fault_part = 122
def plot_feature(feature, tools=all_tools, **plt_kwargs):
    """
    Plot a derived time series feature versus partcount
    Each tool use period is overlayed as a separate line
    Features are automatically normalized to the median per-tool
    """
    dft = df_features.swaplevel()
    fig, ax = plt.subplots(figsize=(24, 5))
    for T in tools:
        (dft.loc[T, feature] / dft.loc[T, feature].median()).plot(ax=ax, label=f'tool {T}', **plt_kwargs)
        ax.set_xlabel(f'partcount')
        ax.set_ylabel('value / (tool median)', fontsize=20)
    ax.set_title(feature)
    plt.gca().relim()
    plt.legend()
    plt.axvline(fault_part, color='red', linewidth=3)
    plt.show()

#plot_feature('duration', tools=[1, 2, 3], ylim=[0.98, 1.02])
plot_feature('load_integral', tools=[1, 2, 3], ylim=[0.95, 1.05])

## 8. Let's dive a bit deeper: Instead of looking at the overall intensity of machining, we can look at its overall *stability*
* Do this by looking at the "noise" on top of the signals, in both time-domain and frequency-domain (Fourier transform).
* To get a clear noise signal, we will first de-trend the data at the 100-millisecond scale. (Subtract a smoothed-out signal.)
* Spindle speed requires simpler processing than spindle load since it's mostly flat, so let's focus on that metric.

In [ ]:
smoothing_window = 101

test_part, test_tool = 100, 1
tmin, tmax = 1000, 5000

fig, ax = plt.subplots(figsize=(24, 5))
speed = get_part_tool(test_part, test_tool)['speed'].iloc[tmin:tmax]
speed_smoothed = speed.rolling(smoothing_window, center=True).mean()
speed.plot(use_index=False, label='original signal')
speed_smoothed.plot(use_index=False, linewidth=4, label='smoothed signal')
plt.xlabel('milliseconds')
plt.ylabel('Speed (RPM)')
plt.legend()
plt.show()

## 9. Scanning the time and frequency signals near the fault reveals something interesting
* The tool that failed is mostly operating at 2100 RPM, which corresponds to 35 Hz. That should be an interesting place to monitor for anomalies in frequency space.
* Pay attention to when part #106 comes up!

In [ ]:
from scipy.signal import periodogram

smoothing_window = 101

test_parts, test_tool = range(95, 122+1), 1

xlim_time = 0, 41000
ylim_time = -4, 4
xlim_freq = 0, (2100 / 60) * 2.5
ylim_freq = 0, 0.3

def get_masked_speed_noise(partcount, T):
    """
    Subtract smoothed speed signal from original speed signal to get the "noise"
    Further, mask any regions near where the spindle is rapidly accelerating
    """
    speed = get_part_tool(partcount, T)['speed']
    if (len(speed) == 0):
        return speed
    speed_smoothed = speed.rolling(smoothing_window, center=True).mean()
    mask = (
        (speed_smoothed.diff().abs() > 1)
        .rolling(2 * smoothing_window + 1, center=True).sum()
        .fillna(1)
        .map(bool)
    )
    return (speed - speed_smoothed).mask(mask)

for partcount in test_parts:
    if (partcount, test_tool) not in all_parts_tools:
        continue
    
    t_start = time()
    speed_noise = get_masked_speed_noise(partcount, test_tool)
    freqs, power = periodogram(speed_noise.dropna(), fs=1000)
    
    clear_output(wait=True)
    fig, axs = plt.subplots(2, 1, figsize=(20,10))
    
    ax = axs[0]
    speed_noise.plot(ax=ax, use_index=False, linewidth=0.5)
    ax.set_title(f'Part {partcount},  Tool {test_tool}', fontsize=20, fontweight='bold')
    ax.set_xlim(xlim_time)
    ax.set_ylim(ylim_time)
    ax.set_xlabel('Time (milliseconds)')
    ax.set_ylabel('Speed noise (RPM)')
    
    ax = axs[1]
    ax.plot(freqs, power, linewidth=1)
    ax.set_xlim(xlim_freq)
    ax.set_ylim(ylim_freq)
    ax.set_xlabel('Frequency (Hz)')
    ax.set_ylabel('Power density (arb. units)')
    
    plt.subplots_adjust(hspace=0.35)
    plt.show()
    t_elapsed = time() - t_start
    sleep(max(0, 0.5 - t_elapsed))

## 10. Now construct new time series features based on this observation
We'll just integrate the spectral power over a handful of frequency bands:
* "low-frequency": 0 - 100 Hz
* "medium-frequency": 100 - 250 Hz
* "high-frequency": 250 - 500 Hz

In [ ]:
new_feature_rows = []
previous_partcount = 0
for partcount, T in all_parts_tools:
    if partcount != previous_partcount:
        print(partcount, end=' ')
        previous_partcount = partcount
    speed_noise = get_masked_speed_noise(partcount, T)
    freqs, power = periodogram(speed_noise.dropna(), fs=1000)
    new_feature_rows.append({
        'power_low_freq' :power[(  0 <= freqs) & (freqs < 100)].sum(),
        'power_med_freq' :power[(100 <= freqs) & (freqs < 250)].sum(),
        'power_high_freq':power[(250 <= freqs) & (freqs < 501)].sum(),
        })
print()

df_new_features = pd.DataFrame(index=all_parts_tools, data=new_feature_rows)
df_features = df_features.join(df_new_features)

plot_feature('power_low_freq')
plot_feature('power_med_freq')
plot_feature('power_high_freq')

## 11. *Bonus*: Using a spectrogram pinpoints noise anomalies simultaneously in frequency and time
What other ways can we think of to apply digital signal processing methods to reveal aspects of impending tool failures?

In [ ]:
from scipy.signal import spectrogram
import numpy as np

test_part, test_tool = 100, 1 
fig, ax = plt.subplots(figsize=(20,5))
speed_noise = get_masked_speed_noise(test_part, test_tool)
f, t, power = spectrogram(speed_noise.dropna(), 1000, nfft=2000)
heatmap_kwargs = {'cmap':'RdYlBu_r', 'xticklabels':10, 'cbar_kws':{'label': 'arb. units'}}
sns.heatmap(
    pd.DataFrame(index=f, columns=t, data=np.log(power)).loc[:250].iloc[::-1], 
    vmin=-20, vmax=-3, **heatmap_kwargs
)
plt.xlabel('Time (seconds)')
plt.ylabel('Frequency (Hz)')
plt.title(f'Noise log-spectrogram of a normal part & tool (Part {test_part}, Tool {test_tool})')
plt.show()

power_ref = power

test_part, test_tool = 107, 1 
fig, ax = plt.subplots(figsize=(20,5))
speed_noise = get_masked_speed_noise(test_part, test_tool)
f, t, power = spectrogram(speed_noise.dropna(), 1000, nfft=2000)
sns.heatmap(
    pd.DataFrame(index=f, columns=t, data=(power - power_ref)).loc[0:250].iloc[::-1], 
    vmin=-0.03, vmax=0.1, **heatmap_kwargs
)
plt.xlabel('Time (seconds)')
plt.ylabel('Frequency (Hz)')
plt.title(f'Noise excess for an early faulty part (Part {test_part}, Tool {test_tool})')
plt.show()

test_part, test_tool = 121, 1 
fig, ax = plt.subplots(figsize=(20,5))
speed_noise = get_masked_speed_noise(test_part, test_tool)
f, t, power = spectrogram(speed_noise.dropna(), 1000, nfft=2000)
sns.heatmap(
    pd.DataFrame(index=f, columns=t, data=(power - power_ref)).loc[0:250].iloc[::-1], 
    vmin=-0.03, vmax=0.1, **heatmap_kwargs
)
plt.xlabel('Time (seconds)')
plt.ylabel('Frequency (Hz)')
plt.title(f'Noise excess for a later faulty part (Part {test_part}, Tool {test_tool})')
plt.show()

## See our blog post on this topic! https://www.machinemetrics.com/techblog/the-inaugural-machinemetrics-8-track